# Your implementation task



1. Implement the EM-algorithm to find a Gaussian NBC for the digits dataset from SciKitLearn (you can of course also use the MNIST_Light set from Lab 5, but for initial tests the digits data set is more convenient, since it is smaller in various aspects). You may assume (conditional) independence between the attributes, i.e., the covariances can be assumed to be simply the variances over each attribute. Split the data set in 70% training and 30% test data.


### SciKitLearn Digits <a name='sklearn1'></a>
Split the data set in 70% training and 30% test data.

In [138]:
from sklearn.datasets import load_digits
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

min_max_scaler = MinMaxScaler()
X, y = load_digits(n_class=10, return_X_y=True)
# I have to do this to get reasonable results
X= min_max_scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3, 
                                                    random_state=42)

In [76]:
X_train.shape


(1257, 64)

### EM-algorithm

<img src="E_step.png">

<img src="M_step.png">

In [139]:
from scipy.stats import norm
import numpy as np
import sys


def EM_algorithm(X, k_classes, N=200,  eps=5e-2):
    
    def E_step(X, k_classes, mu, sig, prior):
        r = np.zeros([len(X), k_classes]) # Shape 1257, 10
        for i, xi in enumerate(X):
            # I used the predefined norm.pdf function from scipy
            num = np.prod([norm.pdf(xi, mu[kcl], np.sqrt(sig[kcl])) for kcl in range(k_classes)], axis=1)
            # Here I just multiply by the prior, pi_k
            num = [prior[kcl]*num[kcl] for kcl in range(k_classes)]
            den = np.sum(num)
            r[i, :] = num/den            
        return r 

    def M_step(X, r):
        N, k_classes = r.shape
        # This is just pi_k^t from the formula above but for all k's, i.e. just pi^t
        prior = {kcl:np.sum(r[:, kcl])/N for kcl in range(k_classes)}
        mu = {}
        sig = {}
        
        for kcl in range(k_classes):
            mu[kcl] = np.sum([r[i, kcl]*xi for i, xi in enumerate(X)], axis=0)/np.sum(r[:, kcl])
            # I take the diag since we only have a variance (no covariance since independet variables)
            sig[kcl] = np.sum([r[i, kcl]*np.diag(np.outer(xi, xi)) for i, xi in enumerate(X)], axis= 0)/np.sum(r[:, kcl]) - np.diag(np.outer(mu[kcl], mu[kcl])) + eps
        return mu, sig, prior
    
    
    # Initial values
    mu = {kcl:np.random.uniform(1, size=X[0].size) for kcl in range(k_classes)}
    sig = {kcl:np.random.uniform(0.5, size=X[0].size) for kcl in range(k_classes)}
    prior = {kcl:1/k_classes for kcl in range(k_classes)} # This is pi_k in the notes                                                    
    
    # Here I iterate the E and M step a fixed amount of times (not necessarily until convergence)
    for iteration in range(N):
        r = E_step(X_train, k_classes, mu, sig, prior)
        mu, sig, prior = M_step(X_train, r)
        
    class GNB_classifier:
        def __init__(self, mu, sig, prior, k_classes, eps):
            self.mu = mu
            self.sig = sig
            self.prior = prior
            self.eps = eps
            self.classes = k_classes
        
        
        def predict(self, X):
            # This is all taken from lab 5 from the Gaussian Naive Bayesian Classifier (GNB) class
            y_pred = []
            for x in X:
                prob = [self._posterior(x, kcl) for kcl in range(self.classes)]
                y_pred.append(np.argmax(prob))
            return np.asarray(y_pred)
    
    
        def _posterior(self, x, ci):
            return self.prior[ci]*np.prod([norm.pdf(x[i], self.mu[ci][i], self.sig[ci][i] + self.eps) for i in range(len(x))])
    
    return GNB_classifier(mu, sig, prior, k_classes, eps)

In [115]:
import numpy as np
from scipy.stats import norm

# Fuck this
class EM_gNBC:

    def __init__(self, num_classes, shape): # shape should be a tuple
        self.num_classes = num_classes
        self.gaussian = {cl: {'mean': np.random.uniform(low=0, high=1, size=shape),
                              'var': np.random.uniform(low=0, high=1, size=shape)} for cl in range(num_classes)}
        
        self.prior = {cl: np.random.uniform(low=0, high=1, size=1) for cl in range(num_classes)} # This is pi_k in the notes                                                    
    
    def E_step(self, X_train):
        # Den här borde jag nog flytta till __init__()
        r = np.zeros((X_train.shape[0], self.num_classes)) # Shape 1257, 10
        
        for index, x in enumerate(X_train):
            num = np.prod([norm.pdf(x, self.gaussian[k]['mean'], np.sqrt(self.gaussian[k]['var'])) 
                                for k in range(self.num_classes)], axis= 1)
            num = [self.prior[k]*num[k] for k in range(num_classes)]
            num = np.asarray(num)
            print(num.shape)
            den = np.sum(num)
            print(den.shape)
            a = num/den
            print(a.shape)
            r[index,:] = a
        return r
    
    def M_step(self, X_train, ri):
        #r = np.sum(ri)
        
        return none
        
        
        

In [ ]:
 r = np.zeros((len(X_train), self.num_classes))
            
            for index,x in enumerate(X_train):
                num = np.prod([norm.pdf(x, self.gaussian[k]['mean'], np.sqrt(self.gaussian[k]['var']))
                                    for k in range(self.num_classes)], axis = 1)
                num = [self.priors[k]*num[k] for k in range(self.num_classes)]
                den = np.sum(num)
                r[index,:] = num/den

In [116]:
num_classes = 10
shape = X_train.shape
model = EM_gNBC(num_classes, shape)

r = model.E_step(X_train)

(10, 64)
()
(10, 64)


/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in true_divide


ValueError: could not broadcast input array from shape (10,64) into shape (10)

In [68]:
import numpy as np

test = {cl: [1, 2, 3] for cl in range(10)}
test[0]

#shape = (5,2)
#a = np.random.uniform(low=0, high=1, size=shape)

#print(a)
np.sqrt(np.pi)


for index, val in enumerate(X_train):
    print('index', index,'val:', val)
    if i == 4:
        break
    i +=1

index 0 val: [ 0.  0.  5. 13. 13.  8.  0.  0.  0.  0. 16. 11. 13. 16.  6.  0.  0.  1.
 16.  5.  2. 14.  9.  0.  0.  0.  9. 16. 16. 15.  0.  0.  0.  0. 10. 16.
 14. 14.  0.  0.  0.  5. 15.  4.  0. 16.  6.  0.  0.  6. 14.  7.  6. 16.
  4.  0.  0.  0.  7. 15. 16. 10.  0.  0.]
<class 'numpy.ndarray'>
index 1 val: [ 0.  0.  3. 14. 16. 14.  0.  0.  0.  0. 13. 13. 13. 16.  2.  0.  0.  0.
  1.  0.  9. 15.  0.  0.  0.  0.  9. 12. 15. 16. 10.  0.  0.  4. 16. 16.
 16. 11.  3.  0.  0.  0.  4.  9. 14.  2.  0.  0.  0.  0.  2. 15.  9.  0.
  0.  0.  0.  0.  4. 13.  1.  0.  0.  0.]
<class 'numpy.ndarray'>
index 2 val: [ 0.  0.  5. 13.  2.  0.  0.  0.  0.  0.  4. 16.  7.  0.  0.  0.  0.  0.
  4. 16.  4.  0.  0.  0.  0.  0.  4. 16.  6.  0.  0.  0.  0.  0.  9. 16.
 10.  0.  0.  0.  0.  0.  2. 11. 15.  1.  0.  0.  0.  0. 10. 13. 16. 15.
 16.  9.  0.  0.  3. 12. 16. 16. 11.  2.]
<class 'numpy.ndarray'>
index 3 val: [ 0.  0.  0.  6. 16.  2.  0.  0.  0.  0.  2. 15. 15.  0.  0.  0.  0.  0.
 15. 16.  3.  2.  3.

2. Use the result of the EM-algorithm (the found distribution parameters) to cluster the training data (essentially, using the resulting classifier to do a prediction over the training data). Produce a confusion matrix over the known labels for the training data and your EM-generated clusters. What do you see?


*Homogeneity: score between 0.0 and 1.0. 1.0 stands for perfectly homogeneous labeling*

*Completeness: score between 0.0 and 1.0. 1.0 stands for perfectly complete labeling*

*V_measure: harmonic mean of the first two*

In [140]:
from sklearn import metrics

gnb_clf = EM_algorithm(X_train, k_classes=10, N=200)
gnb_pred = gnb_clf.predict(X_test)
measure = metrics.homogeneity_completeness_v_measure(y_test, gnb_pred)

print("Homogeneity Score: ", measure[0])
print("Completeness Score: ", measure[1])
print("V-measure: ", measure[2])

Homogeneity Score:  0.7426691658751972
Completeness Score:  0.7661825944183708
V-measure:  0.7542426675422047


3. If necessary, find a way to "repair" the cluster assignments so that you can do a prediction run over the test data, from which you can compare the results with your earlier implementation of the Gaussian NBC.


In [141]:
# I definitevly need to repair my cluster assignment judging from the results.
print("Classification report GNB:\n%s\n"
      % (metrics.classification_report(y_test, gnb_pred)))
print("Confusion matrix GNB:\n%s" % metrics.confusion_matrix(y_test, gnb_pred))

Classification report GNB:
              precision    recall  f1-score   support

           0       1.00      0.94      0.97        53
           1       0.00      0.00      0.00        50
           2       0.00      0.00      0.00        47
           3       0.00      0.00      0.00        54
           4       0.95      0.95      0.95        60
           5       0.00      0.00      0.00        66
           6       0.00      0.00      0.00        53
           7       0.06      0.05      0.06        55
           8       0.00      0.00      0.00        43
           9       0.05      0.03      0.04        59

    accuracy                           0.21       540
   macro avg       0.21      0.20      0.20       540
weighted avg       0.22      0.21      0.21       540


Confusion matrix GNB:
[[50  0  1  0  2  0  0  0  0  0]
 [ 0  0  0  0  0 13  0 27 10  0]
 [ 0  0  0  0  0  0  0  0 41  6]
 [ 0 42  0  0  0  0  2  2  0  8]
 [ 0  0  0  0 57  1  2  0  0  0]
 [ 0 20 44  1  1  0  0  0 

In [142]:
def repair(X_train, y_train, y_pred):
    k_classes = list(set(y_train))
    c_true = {}
    c_pred = {}
    for kcl in k_classes:
        c_true[kcl] = np.mean([X for i, X in enumerate(X_train) if y_train[i] == kcl], axis=0)
        c_pred[kcl] = np.mean([X for i, X in enumerate(X_train) if y_pred[i] == kcl], axis=0)

    pred2true = {}
    for kcl in k_classes:
        pred2true[kcl] = np.argmin([np.linalg.norm(c_pred[kcl] - c_true_v) for c_true_v in c_true.values()])
    
    y_new = []
    for y in y_pred:
        y_new.append(pred2true[y])
    return np.asarray(y_new)


def repair_ind(X_train, y_train, y_pred):
    k_classes = list(set(y_train))
    pred2true = {}
    for kcl in k_classes:
        indices = [i for i, x in enumerate(X_train) if y_pred[i] == kcl]
        unique, counts= np.unique(y_train[indices], return_counts=True)
        pred2true[kcl] = unique[np.argmax(counts)]
        
    y_new = []
    for y in y_pred:
        y_new.append(pred2true[y])
    return np.asarray(y_new)

In [157]:
gnb_pred_fixed = repair_ind(X_test, y_test, gnb_pred)

measure = metrics.homogeneity_completeness_v_measure(y_test, gnb_pred_fixed)

print("Homogeneity Score: ", measure[0])
print("Completeness Score: ", measure[1])
print("V-measure: ", measure[2])
print('\n')
print("Classification report:\n%s\n"
      % (metrics.classification_report(y_test, gnb_pred_fixed)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(y_test, gnb_pred_fixed))

Homogeneity Score:  0.7316801857859325
Completeness Score:  0.7818801149280263
V-measure:  0.7559476652274862


Classification report:
              precision    recall  f1-score   support

           0       1.00      0.94      0.97        53
           1       0.56      0.80      0.66        50
           2       0.80      0.87      0.84        47
           3       0.00      0.00      0.00        54
           4       0.95      0.95      0.95        60
           5       0.94      0.67      0.78        66
           6       0.98      1.00      0.99        53
           7       0.88      0.95      0.91        55
           8       0.57      0.49      0.53        43
           9       0.43      0.80      0.56        59

    accuracy                           0.75       540
   macro avg       0.71      0.75      0.72       540
weighted avg       0.72      0.75      0.72       540


Confusion matrix:
[[50  0  0  0  2  1  0  0  0  0]
 [ 0 40 10  0  0  0  0  0  0  0]
 [ 0  0 41  0  0  0  

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Now I got reasonable results

4. Use now also the k-Means implementation from SciKitLearn and compare the results to yours (they should be similar at least in the sense that there are classes that are more clearly separated from the rest than others for both approaches). 

In [156]:
from sklearn.cluster import KMeans

km_clf = KMeans(n_clusters=10, random_state=42).fit(X_train)
km_pred = km_clf.predict(X_test)
km_pred = repair(X_test, y_test, km_pred)

measure = metrics.homogeneity_completeness_v_measure(y_test, km_pred)

print("Homogeneity Score: ", measure[0])
print("Completeness Score: ", measure[1])
print("V-measure: ", measure[2])
print('\n')
print("Classification report:\n%s\n"
      % (metrics.classification_report(y_test, km_pred)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(y_test, km_pred))

Homogeneity Score:  0.7561055338287734
Completeness Score:  0.8081139384915079
V-measure:  0.7812451278991431


Classification report:
              precision    recall  f1-score   support

           0       0.98      0.98      0.98        53
           1       0.77      0.80      0.78        50
           2       0.79      0.87      0.83        47
           3       0.00      0.00      0.00        54
           4       0.96      0.90      0.93        60
           5       0.94      0.68      0.79        66
           6       0.96      0.98      0.97        53
           7       0.87      1.00      0.93        55
           8       0.82      0.86      0.84        43
           9       0.41      0.81      0.55        59

    accuracy                           0.79       540
   macro avg       0.75      0.79      0.76       540
weighted avg       0.75      0.79      0.76       540


Confusion matrix:
[[52  0  0  0  1  0  0  0  0  0]
 [ 0 40 10  0  0  0  0  0  0  0]
 [ 0  0 41  0  0  0  

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
